In [45]:
import os
import yaml
from torch.utils.data import DataLoader
import argparse

from GeospatialFM.data import get_datasets
from GeospatialFM.models import *
# from utils import load_config
from torchgeo.samplers import RandomGeoSampler
from matplotlib import pyplot as plt

from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup
from GeospatialFM.utils import setup, get_eval_fn, get_data
from GeospatialFM.data import *
from GeospatialFM.models import *
from GeospatialFM.loss import *

from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import numpy as np
from torch.utils.data import ConcatDataset
import segmentation_models_pytorch as smp

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
device = torch.device('cuda:0')

In [6]:
args = {'exp_name': None,
        'config_file': 'GeospatialFM/configs/mae.yaml',
        'opts': None, 
        'save_config': False}
args = argparse.Namespace(**args)
args.debug = True
args

Namespace(exp_name=None, config_file='GeospatialFM/configs/mae.yaml', opts=None, save_config=False, debug=True)

In [7]:
cfg, _ = setup(args)

In [9]:
data = get_data(cfg)
data

{'train': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x7fbc94696890>, sampler=None, shared_epoch=None),
 'val': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x7fbc946965f0>, sampler=None, shared_epoch=None),
 'test': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x7fbc94696230>, sampler=None, shared_epoch=None)}

In [39]:
sample = iter(data['train'].dataloader).__next__()

In [40]:
sample['image'].shape

torch.Size([256, 13, 224, 224])

In [38]:
mae = construct_mae(cfg.MODEL)

In [41]:
ret = mae(sample['image'], sample['radar'])

In [48]:
mae_loss = MAELoss()
mmce_loss = MultiModalCELoss()

In [47]:
mae_loss(**ret, return_dict=True)

{'optical_mse': tensor(0.8039, grad_fn=<DivBackward0>),
 'radar_mse': tensor(0.8062, grad_fn=<DivBackward0>)}

In [51]:
mmce_loss(**ret, labels=sample['label'], return_dict=True)


{'optical_ce': tensor(2.9425, grad_fn=<NllLossBackward0>),
 'radar_ce': tensor(2.8541, grad_fn=<NllLossBackward0>)}